# Advanced processing of Strands Agents Response

Strands Agents allows you to intercept and process events as they happen during agent execution using two methods: 
    
- **Async iterators**: ideal for asynchronous frameworks like FastAPI, aiohttp, or Django Channels. For these environments, the SDK offers the `stream_async` method which returns an asynchronous iterator. 
- **Callback handlers**: allow you to intercept and process events as they happen during agent execution. This enables real-time monitoring, custom output formatting, and integration with external systems.

In this example, we will show you how to use both methods to handle calls on your agent


## Agent Details
<div style="float: left; margin-right: 20px; ">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |async iterators, callback handlers                 |
|Agent Structure     |single agent architecture                          |
|Native tools used   |calculator                                         |
|Custom tools created|Weather forecast                                   |

</div>

## Architecture

<div style="text-align:left;">
    <img src="./images/architecture.png" width="65%" />
</div>

## Key Features
* Async Iterators for Streaming
* Callback Handlers


## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent Agent

In [ ]:
# installing pre-requisites
# !pip install -r requirements.txt

### Importing dependency packages

Now let's import the dependency packages

In [25]:
import asyncio

import httpx
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
from strands import Agent, tool
from strands_tools import calculator

## Method 1 - Async Iterators for Streaming


Strands Agents provides support for asynchronous iterators through the `stream_async` method, enabling real-time streaming of agent responses in asynchronous environments like web servers, APIs, and other async applications.

Since we are show casing this example in a notebook, we need to apply `nest_asyncio` to allow nested use of `asyncio.run` and `loop.run_until_complete`

In [26]:
nest_asyncio.apply()

### Creating and invoking agent with stream_async

Let's now create our agent with a built-in calculator tool and no `callback_handler`. We will use the `stream_async` method to iteract over the streamed agent events

In [27]:
# Initialize our agent without a callback handler
# model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
# model = BedrockModel(
#     model_id=model_id,
# )
agent = Agent(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",  # Optional: Specify the model ID
    tools=[calculator], 
    callback_handler=None)

# Async function that iterators over streamed agent events


async def process_streaming_response():
    agent_stream = agent.stream_async("Calculate 2+2")
    async for event in agent_stream:
        print(event)


# Run the agent
asyncio.run(process_streaming_response())

{'init_event_loop': True}
{'start': True}
{'start_event_loop': True}
{'event': {'messageStart': {'role': 'assistant'}}}
{'event': {'contentBlockDelta': {'delta': {'text': 'I'}, 'contentBlockIndex': 0}}}
{'data': 'I', 'delta': {'text': 'I'}, 'agent': <strands.agent.agent.Agent object at 0x74c0cda49550>, 'event_loop_cycle_id': UUID('353ddf91-4737-4068-bdb8-24c209ecfa46'), 'request_state': {}, 'event_loop_cycle_trace': <strands.telemetry.metrics.Trace object at 0x74c0cd0d1810>, 'event_loop_cycle_span': NonRecordingSpan(SpanContext(trace_id=0x00000000000000000000000000000000, span_id=0x0000000000000000, trace_flags=0x00, trace_state=[], is_remote=False))}
{'event': {'contentBlockDelta': {'delta': {'text': "'ll"}, 'contentBlockIndex': 0}}}
{'data': "'ll", 'delta': {'text': "'ll"}, 'agent': <strands.agent.agent.Agent object at 0x74c0cda49550>, 'event_loop_cycle_id': UUID('353ddf91-4737-4068-bdb8-24c209ecfa46'), 'request_state': {}, 'event_loop_cycle_trace': <strands.telemetry.metrics.Trace o

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 2+2                 │                                                                            │
│  │ Result    │ 4                   │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'message': {'role': 'user', 'content': [{'toolResult': {'status': 'success', 'content': [{'text': 'Result: 4'}], 'toolUseId': 'tooluse_NYGZOgJoQUeyZaoY0ggJzQ'}}]}}
{'start': True}
{'start': True}
{'start_event_loop': True}
{'event': {'messageStart': {'role': 'assistant'}}}
{'event': {'contentBlockDelta': {'delta': {'text': 'The result'}, 'contentBlockIndex': 0}}}
{'data': 'The result', 'delta': {'text': 'The result'}, 'agent': <strands.agent.agent.Agent object at 0x74c0cda49550>, 'event_loop_cycle_id': UUID('059adafe-f7e9-429c-9a0e-ad896f594eb3'), 'request_state': {}, 'event_loop_cycle_trace': <strands.telemetry.metrics.Trace object at 0x74c0cd0d2210>, 'event_loop_cycle_span': NonRecordingSpan(SpanContext(trace_id=0x00000000000000000000000000000000, span_id=0x0000000000000000, trace_flags=0x00, trace_state=[], is_remote=False)), 'model': <strands.models.bedrock.BedrockModel object at 0x74c0d934aa50>, 'messages': [{'role': 'user', 'content': [{'text': 'Calculate 2+2'}]}, {'role': 'assi

###  Tracking event loop lifecycle

This example illustrates the event loop lifecycle and how events relate to each other. It's useful for understanding the flow of execution in the Strands Agent:

Let's create some printing format code to better analyse the agent stream events. We will continue to use the same agent for it

In [28]:
# Async function that iterators over streamed agent events


async def process_streaming_response():
    agent_stream = agent.stream_async("What is the capital of France and what is 42+7?")
    async for event in agent_stream:
        # Track event loop lifecycle
        if event.get("init_event_loop", False):
            print("🔄 Event loop initialized")
        elif event.get("start_event_loop", False):
            print("▶️ Event loop cycle starting")
        elif event.get("start", False):
            print("📝 New cycle started")
        elif "message" in event:
            print(f"📬 New message created: {event['message']['role']}")
        elif event.get("force_stop", False):
            print(
                f"🛑 Event loop force-stopped: {event.get('force_stop_reason', 'unknown reason')}"
            )

        # Track tool usage
        if "current_tool_use" in event and event["current_tool_use"].get("name"):
            tool_name = event["current_tool_use"]["name"]
            print(f"🔧 Using tool: {tool_name}")

        # Show only a snippet of text to keep output clean
        if "data" in event:
            # Only show first 20 chars of each chunk for demo purposes
            data_snippet = event["data"][:20] + (
                "..." if len(event["data"]) > 20 else ""
            )
            print(f"📟 Text: {data_snippet}")

    return event["result"]


# Run the agent
asyncio.run(process_streaming_response())

🔄 Event loop initialized
📝 New cycle started
▶️ Event loop cycle starting


📟 Text: I
📟 Text:  can
📟 Text:  help you with
📟 Text:  the
📟 Text:  mathematical
📟 Text:  calculation
📟 Text: ,
📟 Text:  but I don
📟 Text: 't have
📟 Text:  a
📟 Text:  tool
📟 Text:  available
📟 Text:  to look
📟 Text:  up geographical
📟 Text:  information
📟 Text:  like
📟 Text:  capitals
📟 Text:  of
📟 Text:  countries.
📟 Text: 

Let
📟 Text:  me
📟 Text:  calculate
📟 Text:  
📟 Text: 42+7 
📟 Text: for you:
🔧 Using tool: calculator
🔧 Using tool: calculator
🔧 Using tool: calculator
🔧 Using tool: calculator
🔧 Using tool: calculator
📬 New message created: assistant


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 42+7                │                                                                            │
│  │ Result    │ 49                  │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

📬 New message created: user
📝 New cycle started
📝 New cycle started
▶️ Event loop cycle starting
📟 Text: 42
📟 Text:  + 7
📟 Text:  = 49
📟 Text: 

As for the capital...
📟 Text:  - it
📟 Text: 's Paris,
📟 Text:  but I know
📟 Text:  this from
📟 Text:  my general
📟 Text:  knowledge. I
📟 Text:  don
📟 Text: 't have a specific
📟 Text:  tool to verify
📟 Text:  or
📟 Text:  look up geographica...
📟 Text:  information
📟 Text: .
📬 New message created: assistant


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "42 + 7 = 49\n\nAs for the capital of France - it's Paris, but I know this from my general knowledge. I don't have a specific tool to verify or look up geographical information."}]}, metrics=EventLoopMetrics(cycle_count=4, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'tooluse_eFb_Q3oPQKqAD9dhLXHyWw', 'name': 'calculator', 'input': {'expression': '42+7'}}, call_count=2, success_count=2, error_count=0, total_time=0.43875694274902344)}, cycle_durations=[1.0506322383880615, 1.9413537979125977], traces=[<strands.telemetry.metrics.Trace object at 0x74c0cd0d1810>, <strands.telemetry.metrics.Trace object at 0x74c0cd0d2210>, <strands.telemetry.metrics.Trace object at 0x74c0cfa44370>, <strands.telemetry.metrics.Trace object at 0x74c0cd0d1a90>], accumulated_usage={'inputTokens': 8215, 'outputTokens': 228, 'totalTokens': 8443}, accumulated_metrics={'latencyMs': 7273}), state={})

### FastAPI Integration

You can also integrate your `stream_async` with FastAPI to create a streaming endpoint to your applications. For this, we will add a `weather_forecast` tool to our agent. The architecture update looks as following

<div style="text-align:left;">
    <img src="./images/architecture_2.png" width="65%" />
</div>

In [29]:
# # Tool definition
from strands.models import BedrockModel
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
model = BedrockModel(
    model_id=model_id,
)

@tool
def weather_forecast(city: str, days: int = 3) -> str:
    return f"Weather forecast for {city} for the next {days} days..."


# FastAPI app
app = FastAPI()


class PromptRequest(BaseModel):
    prompt: str


@app.post("/stream")
async def stream_response(request: PromptRequest):
    async def generate():
        agent = Agent( model = model, 
                      tools=[calculator, weather_forecast], callback_handler=None)
        try:
            async for event in agent.stream_async(request.prompt):
                if "data" in event:
                    yield event["data"]
        except Exception as e:
            yield f"Error: {str(e)}"

    return StreamingResponse(generate(), media_type="text/plain")


# Function to start server without blocking


async def start_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8001, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()


# Run server as background task
if "server_task" not in globals():
    server_task = asyncio.create_task(start_server())
    await asyncio.sleep(0.1)  # Give server time to start

print("✅ Server is running at http://0.0.0.0:8001")

✅ Server is running at http://0.0.0.0:8001


#### Invoking the FastAPI agent
And we can now invoke the agent with a prompt 

In [30]:
async def fetch_stream():
    async with httpx.AsyncClient() as client:
        async with client.stream(
            "POST",
            "http://0.0.0.0:8001/stream",
            json={"prompt": "What is weather in NYC?"},
        ) as response:
            async for line in response.aiter_lines():
                if line.strip():  # Skip empty lines
                    print("Received:", line)


await fetch_stream()

Received: Error: An error occurred (AccessDeniedException) when calling the ConverseStream operation: Model access is denied due to User: arn:aws:sts::381491838394:assumed-role/AWSReservedSSO_AWSAdministratorAccess_afee0d6ba2646b4f/PA.285833D@curtin.edu.au is not authorized to perform: aws-marketplace:Subscribe on resource: * with an explicit deny in a service control policy. Your AWS Marketplace subscription for this model cannot be completed at this time. If you recently fixed this issue, try again after 15 minutes.


## Method 2 - Callback Handlers for streaming

Callback handlers are a powerful feature of the Strands Agents that allow you to intercept and process events as they happen during agent execution. This enables real-time monitoring, custom output formatting, and integration with external systems.



Callback handlers receive events in real-time as they occur during an agent's lifecycle:

- Text generation from the model
- Tool selection and execution
- Reasoning process
- Errors and completions


Let's now create a custom callback handler function that formats the event inputs to highlight tool usage and model output. To do so, we will again use the agent with a calculator tool only

<div style="text-align:left;">
    <img src="images/architecture.png" width="65%" />
</div>

In [ ]:
def custom_callback_handler(**kwargs):
    # Process stream data
    if "data" in kwargs:
        print(f"MODEL OUTPUT: {kwargs['data']}")
    elif "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
        print(f"\nUSING TOOL: {kwargs['current_tool_use']['name']}")


# Create an agent with custom callback handler
agent = Agent(   
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
        tools=[calculator], 
        callback_handler=custom_callback_handler)

agent("Calculate 2+2")

### Congratulations!

In this notebook you learned how to stream your agents outputs using async iteractors and callback handlers. 